<a target="_blank" href="https://colab.research.google.com/github/huoyushequ/TabularTransformer/blob/main/income_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

**See also** [RTDL](https://github.com/yandex-research/rtdl)
-- **other projects on tabular deep learning**.

---

- This notebook provides a usage example of the
  [rtdl_revisiting_models](https://github.com/yandex-research/rtdl-revisiting-models)
  package.
- Hyperparameters are not tuned and may be suboptimal.

In [ ]:
%pip install git+https://github.com/huoyushequ/TabularTransformer.git

In [27]:
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

data_url = "hf://datasets/scikit-learn/adult-census-income/adult.csv"
fname = "income.csv"

data_cache_dir = os.path.join(os.getcwd(), 'data', fname.split('.')[0])
os.makedirs(data_cache_dir, exist_ok=True)
full_path = os.path.join(data_cache_dir, fname)

if not os.path.exists(full_path):
    print(f"Downloading {data_url} to {fname}...")
    df = pd.read_csv(data_url)
    df.to_csv(os.path.join(data_cache_dir, fname))
else:
    df = pd.read_csv(full_path)
    print(f"{full_path} already exists, skipping download.")

/home/qiao/work/TabularTransformer/data/income/income.csv already exists, skipping download.


In [29]:
df.head(3)

,Unnamed: 0,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
